In [112]:
import numpy as np
import pandas as pd
import json
import os
import glob
import re
import nilearn

from nilearn import plotting
from nilearn.signal import clean
import logging


In [113]:
path_derivatives = '../data/derivatives'

path_halfpipe_timeseries = '{}/halfpipe'.format(path_derivatives)
path_fmriprep_confounds = '{}/fmriprep'.format(path_derivatives)

path_atlas = '../data/atlas'

name_atlas = 'schaefer400' ### doit etre aligné avec le nom du fichier de labels, tout dois etre dans path_atlas

output_path = '../draft/output/BEP017'

task = 'task-rest' ### faire une boucle?

spec_path = '../data/spec.json'


In [115]:
path_label_schaefer = '{}/atlas-Schaefer2018Combined_dseg.tsv'.format(path_atlas)

label_schaefer = list(pd.read_csv(path_label_schaefer, sep = '\t', header = None)[1])
print(label_schaefer)


['Schaefer2018_17Networks_LH_VisCent_ExStr_1', 'Schaefer2018_17Networks_LH_VisCent_ExStr_2', 'Schaefer2018_17Networks_LH_VisCent_ExStr_3', 'Schaefer2018_17Networks_LH_VisCent_ExStr_4', 'Schaefer2018_17Networks_LH_VisCent_ExStr_5', 'Schaefer2018_17Networks_LH_VisCent_ExStr_6', 'Schaefer2018_17Networks_LH_VisCent_Striate_1', 'Schaefer2018_17Networks_LH_VisCent_ExStr_8', 'Schaefer2018_17Networks_LH_VisCent_ExStr_9', 'Schaefer2018_17Networks_LH_VisCent_ExStr_10', 'Schaefer2018_17Networks_LH_VisCent_ExStr_11', 'Schaefer2018_17Networks_LH_VisCent_ExStr_12', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_1', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_2', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_3', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_4', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_5', 'Schaefer2018_17Networks_LH_VisPeri_StriCal_1', 'Schaefer2018_17Networks_LH_VisPeri_StriCal_2', 'Schaefer2018_17Networks_LH_VisPeri_ExStrSup_8', 'Schaefer2018_17Networks_LH_VisPeri_ExStrSup_9', 'Sc

In [116]:
len(label_schaefer)


434

In [117]:
sujets = []

for nom in os.listdir(path_halfpipe_timeseries,):
    chemin_complet = os.path.join(path_halfpipe_timeseries, nom)
    if os.path.isdir(path_halfpipe_timeseries,):
        sujets.append(nom)


In [118]:
print(sujets)

['sub-10217', 'sub-10249', 'sub-10159', 'sub-10206', 'sub-10235', 'sub-10225', 'sub-10227', 'sub-10228', 'sub-10189', 'sub-10171']


In [119]:
num_run = '1'


In [120]:
strategies = []

with open(spec_path, "r") as f:
    data = json.load(f)



# Construire un mapping {setting_name: confounds_removal}
setting_to_confounds = {
    s["name"]: s.get("confounds_removal", [])
    for s in data.get("settings", [])
}

# Lier chaque feature à sa stratégie de confounds
strategy_confounds = {}

for feature in data.get("features", []):
    strategy_name = feature.get("name")  # e.g. "corrMatrix3"
    setting_name = feature.get("setting")  # e.g. "corrMatrix3Setting"
    
    confounds = setting_to_confounds.get(setting_name, [])
    strategy_confounds[strategy_name] = confounds

# Affichage du résultat
for strategy, confounds in strategy_confounds.items():
    strategies.append(strategy)
    print(f"Stratégie : {strategy}")
    for c in confounds:
        print(f"  - {c}")

Stratégie : corrMatrix1
  - c_comp_cor_0[0-4]
Stratégie : corrMatrix2
  - (trans|rot)_[xyz]
  - (trans|rot)_[xyz]_derivative1
  - (trans|rot)_[xyz]_power2
  - (trans|rot)_[xyz]_derivative1_power2
  - motion_outlier[0-9]+
Stratégie : corrMatrix3
  - (trans|rot)_[xyz]
  - (trans|rot)_[xyz]_derivative1
  - (trans|rot)_[xyz]_power2
  - (trans|rot)_[xyz]_derivative1_power2
  - motion_outlier[0-9]+
  - global_signal
Stratégie : corrMatrix4
  - (trans|rot)_[xyz]
  - (trans|rot)_[xyz]_derivative1
  - (trans|rot)_[xyz]_power2
  - (trans|rot)_[xyz]_derivative1_power2
Stratégie : corrMatrix5
  - (trans|rot)_[xyz]
  - (trans|rot)_[xyz]_derivative1
  - (trans|rot)_[xyz]_power2
  - (trans|rot)_[xyz]_derivative1_power2
  - global_signal


In [121]:
strategy = strategies[4]


In [122]:
## Cedric work to get all time series

def get_all_halfpipe_timeseries_paths() -> list:
    """Extract all time series TSV path from input directory"""
    INPUT_DIRECTORY = '../data/'
    timeseries_path_pattern = os.path.join(INPUT_DIRECTORY, '**', '*_timeseries.tsv')
    return glob.glob(timeseries_path_pattern, recursive=True)

def extract_timeseries_metadata(files_paths:list) -> tuple:
    """Extract metadata from the file path and return a tuple of subject, session, run, strategy, and atlas"""
    subjects, sessions, runs, strategies, atlases = [], [], [], [], []
    for file_path in files_paths:
        subject_match = re.search(r'/(?:sub-([^/]+))', file_path)
        session_match = re.search(r'/(?:ses-([^/]+))', file_path)
        run_match = re.search(r'/(?:run-([^/]+))', file_path)
        strategy_match = re.search(r'(?:feature-([^_]+))', file_path)
        atlas_match = re.search(r'(?:atlas-([^_]+))', file_path)

        subject = subject_match.group(1) if subject_match else None
        session = session_match.group(1) if session_match else None
        run = run_match.group(1) if run_match else None
        strategy = strategy_match.group(1) if strategy_match else None
        atlas = atlas_match.group(1) if atlas_match else None

        if subject: subjects.append(subject)
        if session: sessions.append(session)
        if run: runs.append(run)
        if strategy: strategies.append(strategy)
        if atlas: atlases.append(atlas)
        
    subjects = list(set(subjects)) 
    sessions = list(set(sessions))
    runs = list(set(runs))
    strategies = list(set(strategies))
    atlases = list(set(atlases))
    return subjects, sessions, runs, strategies, atlases


time_series_files_paths = get_all_halfpipe_timeseries_paths()
subjects, sessions, runs, strategies, atlases = extract_timeseries_metadata(time_series_files_paths)
print(subjects)
print(sessions)
print(runs)
print(strategies) 
print(atlases)


['10159', '10235', '10225', '10227', '10228', '10171', '10189', '10249', '10217', '10206']
[]
[]
['corrMatrix5', 'corrMatrix2', 'corrMatrix3', 'corrMatrix4', 'corrMatrix1']
['schaefer400']


In [123]:
dict_timeseries = {}
dict_samplingfrequency = {}
missing_list = []
dict_mean_framewise = {}
dict_scrubvolume = {}

for sub in sujets:
    
    
    
    # timeseries = '{}/{}/func/task-rest/{}_task-rest_run-{}_feature-{}_atlas-{}_timeseries.tsv'.format(
    #     path_halfpipe_timeseries, sub, sub, num_run, strategy, name_atlas)
    
    timeseries = '{}/{}/func/task-rest/{}_{}_feature-{}_atlas-{}_timeseries.tsv'.format(
        path_halfpipe_timeseries, sub, sub, task, strategy, name_atlas)
    confounds = '{}/{}/func/{}_{}_desc-confounds_timeseries.tsv'.format(
        path_fmriprep_confounds, sub, sub, task)
    timeseries_json = '{}/{}/func/task-rest/{}_{}_feature-{}_atlas-{}_timeseries.json'.format(
        path_halfpipe_timeseries, sub, sub, task, strategy, name_atlas)
    
    try:
        df_timeseries = pd.read_csv(timeseries, sep='\t', header=None)
        df_timeseries.columns = label_schaefer
        dict_timeseries[sub] = df_timeseries

        # TODO : later add the confounds in clean code
        df_confounds = pd.read_csv(confounds, sep='\t')
        mean_framewise = df_confounds['framewise_displacement'].mean()
        dict_mean_framewise[sub] = mean_framewise

        n_motion_outliers = df_confounds.filter(like='motion_outlier').shape[1]
        dict_scrubvolume[sub] = n_motion_outliers

        with open(timeseries_json, "r") as f:
            data = json.load(f)

        sampling_frequency = data.get("SamplingFrequency")
        mean_framewise = data.get("FDMean")

        dict_samplingfrequency[sub] = sampling_frequency


    except FileNotFoundError:
        missing_list.append(sub)
        continue

    

In [124]:
len(missing_list)


0

In [125]:
# Initialisation du compteur de NaNs pour chaque ROI
nan_counts = {label: 0 for label in label_schaefer}
total_subjects = len(dict_timeseries)

# Parcourir tous les sujets
for df in dict_timeseries.values():
    for label in label_schaefer:
        if label in df.columns and df[label].isna().all():
            nan_counts[label] += 1

# Création du DataFrame
df_nan_prop = pd.DataFrame({
    "ROI": list(nan_counts.keys()),
    "proportion_nan": [nan_counts[label] / total_subjects for label in label_schaefer]
})


In [126]:
df_nan_prop.sort_values(by='proportion_nan', ascending=False).reset_index(drop=True)[:30]


,ROI,proportion_nan
0,Buckner2011_17Networks_1,1.0
1,Buckner2011_17Networks_14,1.0
2,Schaefer2018_17Networks_LH_Limbic_TempPole_4,1.0
3,Schaefer2018_17Networks_LH_Limbic_TempPole_1,0.9
4,Schaefer2018_17Networks_RH_Limbic_TempPole_2,0.8
5,Schaefer2018_17Networks_RH_Limbic_TempPole_6,0.8
6,Schaefer2018_17Networks_RH_Limbic_OFC_5,0.8
7,Schaefer2018_17Networks_LH_Limbic_TempPole_2,0.8
8,Schaefer2018_17Networks_RH_Limbic_TempPole_1,0.8
9,Schaefer2018_17Networks_RH_Limbic_TempPole_3,0.7


In [127]:
# Identifier les labels avec une proportion de NaN supérieure à 0.5
labels_a_supprimer = df_nan_prop[df_nan_prop["proportion_nan"] > 0.5]["ROI"].tolist()

# Supprimer ces labels de chaque DataFrame dans dict_timeseries
for key, df in dict_timeseries.items():
    dict_timeseries[key] = df.drop(columns=labels_a_supprimer, errors='ignore')


In [128]:
# Initialiser une liste pour stocker les résultats
liste_proportions = []

# Parcourir chaque sujet et son DataFrame associé
for sujet, df in dict_timeseries.items():
    # Calculer le nombre total de ROI
    total_roi = df.shape[1]
    
    # Compter le nombre de ROI contenant au moins un NaN
    roi_nan = df.isna().any().sum()
    
    # Calculer la proportion de ROI avec des NaN
    proportion_nan = roi_nan / total_roi if total_roi > 0 else 0
    
    # Ajouter les résultats à la liste
    liste_proportions.append({'sujet': sujet, 'proportion_nan': proportion_nan})

# Créer le DataFrame final
df_proportion_nan = pd.DataFrame(liste_proportions)


In [129]:
df_proportion_nan.sort_values(by='proportion_nan', ascending=False).reset_index(drop=True)


,sujet,proportion_nan
0,sub-10217,0.028777
1,sub-10159,0.028777
2,sub-10228,0.028777
3,sub-10171,0.028777
4,sub-10227,0.023981
5,sub-10249,0.019185
6,sub-10206,0.011990
7,sub-10235,0.007194
8,sub-10189,0.007194
9,sub-10225,0.004796


In [130]:
#!TODO: clean code stopped here 

def impute_and_clean(df):
    # Étape 1 : Remplacer les NaN par la moyenne de leur ligne (mean sur l'axe 1)
    row_means = df.mean(axis=1, skipna=True)
    df_filled = df.T.fillna(row_means).T  # on transpose pour appliquer sur les lignes, puis on retranspose

    # Étape 2 : Vérification (optionnel mais utile pour logger ce qu'on fait)
    if df_filled.isna().any().any():
        logging.warning("Certaines valeurs n'ont pas pu être imputées par la moyenne de ligne.")

    # Étape 3 : Nettoyage avec nilearn (standardisation + détrend)
    cleaned = clean(df_filled.values, detrend=True, standardize='zscore_sample')

    # Étape 4 : Retour sous forme de DataFrame
    return pd.DataFrame(cleaned, columns=df.columns, index=df.index)


In [131]:
dict_clean_timeseries = {}
for sub in dict_timeseries.keys():
    dict_clean_timeseries[sub] = impute_and_clean(dict_timeseries[sub])


In [132]:
dict_clean_timeseries[sub]


,Schaefer2018_17Networks_LH_VisCent_ExStr_1,Schaefer2018_17Networks_LH_VisCent_ExStr_2,Schaefer2018_17Networks_LH_VisCent_ExStr_3,Schaefer2018_17Networks_LH_VisCent_ExStr_4,Schaefer2018_17Networks_LH_VisCent_ExStr_5,Schaefer2018_17Networks_LH_VisCent_ExStr_6,Schaefer2018_17Networks_LH_VisCent_Striate_1,Schaefer2018_17Networks_LH_VisCent_ExStr_8,Schaefer2018_17Networks_LH_VisCent_ExStr_9,Schaefer2018_17Networks_LH_VisCent_ExStr_10,...,Buckner2011_17Networks_7,Buckner2011_17Networks_8,Buckner2011_17Networks_9,Buckner2011_17Networks_10,Buckner2011_17Networks_11,Buckner2011_17Networks_12,Buckner2011_17Networks_13,Buckner2011_17Networks_15,Buckner2011_17Networks_16,Buckner2011_17Networks_17
0,-0.662804,-1.693681,-0.645582,-1.080663,-0.759021,-1.130061,0.114203,0.120209,-0.361207,-1.032937,...,-1.537031,-0.622400,-0.125920,-0.544879,0.025576,-0.526935,0.579952,-0.721910,0.634167,0.205169
1,-0.462464,-0.698272,-0.430898,0.978785,1.112590,-0.322936,1.453397,-0.267126,0.063845,-0.160183,...,-1.567337,-0.310417,-0.422603,-0.927546,-0.230239,0.197583,0.913803,-0.409375,1.164735,1.311367
2,0.283338,-0.308986,0.292949,0.681321,0.185974,-0.506794,0.757642,0.238570,-0.430681,-1.352925,...,-0.221431,0.538239,-0.156557,1.771017,0.400724,0.947016,1.341375,-0.925705,0.953198,1.427093
3,0.490735,0.381237,-0.658700,1.055076,-0.368486,-0.260274,0.413734,-0.470906,-0.432458,-1.142728,...,-0.522569,0.445793,0.203192,-1.044712,-0.131298,0.252480,0.621238,0.739841,1.357638,0.981011
4,0.758932,-0.086766,-0.029272,0.428546,1.035678,0.674544,0.265476,-0.295970,0.744315,0.588463,...,-1.423915,0.170002,-0.842407,-0.210067,-0.319092,0.462923,1.731096,-0.291629,2.368330,2.606685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.137906,-1.060996,-0.447442,-0.379416,1.161674,0.241171,-0.215557,-0.204938,0.120343,-0.227203,...,-0.664559,-1.673698,-0.261223,1.484347,-2.337655,-0.106164,-0.000122,-0.976509,0.151942,0.055650
148,-0.206827,0.040251,0.162629,-1.413221,-0.829659,-0.305509,0.258361,0.113468,-0.347161,0.818891,...,-0.586025,-0.511756,-1.851079,-0.256374,-1.280366,-0.781761,-0.748014,-1.217070,-1.618813,-0.104350
149,-0.269875,0.299718,0.818854,-0.386824,-1.120325,-0.713215,-0.747765,0.360990,-0.720095,-0.498134,...,-1.444555,-1.036087,-0.605707,0.726220,-1.374148,-1.251777,-1.522868,0.120632,-1.001597,-1.042934
150,-0.366162,0.643564,-0.412510,-0.441972,0.911167,0.177889,0.323959,-0.631848,0.822067,1.606850,...,-0.139129,-0.340069,0.034660,0.004348,0.209774,0.972217,-0.035722,-0.743554,0.001191,0.472236


In [133]:
volume_path = '{}/atlas-Schaefer2018Combined_dseg.nii.gz'.format(path_atlas)

coords = plotting.find_parcellation_cut_coords(volume_path)


In [134]:
coords.shape


(434, 3)

In [135]:
df_coords = pd.DataFrame(coords, index=label_schaefer, columns=['x', 'y', 'z'])


In [136]:
df_coords


,x,y,z
Schaefer2018_17Networks_LH_VisCent_ExStr_1,-37.286885,-62.519672,-17.122951
Schaefer2018_17Networks_LH_VisCent_ExStr_2,-24.904651,-73.476744,-9.988372
Schaefer2018_17Networks_LH_VisCent_ExStr_3,-38.226496,-81.685185,-15.673789
Schaefer2018_17Networks_LH_VisCent_ExStr_4,-18.255814,-87.476744,-14.587209
Schaefer2018_17Networks_LH_VisCent_ExStr_5,-26.312613,-97.649550,-11.372072
...,...,...,...
Buckner2011_17Networks_13,-34.875911,-70.730917,-36.717108
Buckner2011_17Networks_14,-21.300000,-73.700000,-39.700000
Buckner2011_17Networks_15,-16.932432,-39.283784,-45.918919
Buckner2011_17Networks_16,-7.010251,-52.796128,-44.422551


In [137]:
len(labels_a_supprimer)


17

In [138]:
df_coords = df_coords[~df_coords.index.isin(labels_a_supprimer)]


In [139]:
remaining_labels = list(set(label_schaefer) - set(labels_a_supprimer))


In [140]:
dict_corrpearson_matrix = {}
for sub in dict_clean_timeseries.keys():
    dict_corrpearson_matrix[sub] = dict_clean_timeseries[sub].corr(method='pearson')


In [141]:
name_ds = 'halfpipe_outputs_timeseries_HBN_dev'
fmriprep_version = 'fmriprep-25.0.0'
name_atlas = 'atlas-schaefercombined'
task_name = 'task-rest'
space = 'space-MNI152NLin2009cAsym'

nroi = dict_clean_timeseries[sub].shape[1]

regressors = strategy_confounds[strategy]


In [142]:
for sub in dict_clean_timeseries.keys():

    ### timeseries
    file_name = '{}_{}_seg-{}{}_desc-denoise{}_timeseries.tsv'.format(sub, task_name, name_atlas, nroi, strategy)
    file_output = '{}/{}/func/{}'.format(output_path, sub, file_name)
    os.makedirs(os.path.dirname(file_output), exist_ok=True)
    
    dict_clean_timeseries[sub].columns = range(dict_clean_timeseries[sub].shape[1])
    dict_clean_timeseries[sub].to_csv(file_output, sep="\t", header=True, index=False)

    ### connectomes pearson
    file_name = '{}_{}_seg-{}{}_meas-PearsonCorrelation_desc-denoise{}_relmat.tsv'.format(sub, task_name, name_atlas, nroi, strategy)
    file_output = '{}/{}/func/{}'.format(output_path, sub, file_name)
    os.makedirs(os.path.dirname(file_output), exist_ok=True)
    
    dict_corrpearson_matrix[sub].columns = range(dict_corrpearson_matrix[sub].shape[1])
    dict_corrpearson_matrix[sub].to_csv(file_output, sep="\t", header=True, index=False)

    ### json timeseries
    json_dataset_description = {
        "ConfoundRegressors": regressors,
        "NumberOfVolumesDiscardedByMotionScrubbing": dict_scrubvolume[sub],
        "MeanFramewiseDisplacement": dict_mean_framewise[sub],
        "SamplingFrequency": dict_samplingfrequency[sub]
    }
    
    file_name = '{}_{}_seg-{}{}_desc-denoise{}_timeseries.json'.format(sub, task_name, name_atlas, nroi, strategy)
    file_output = '{}/{}/func/{}'.format(output_path, sub, file_name)

    # Export JSON
    with open(file_output, 'w') as f:
        json.dump(json_dataset_description, f, indent=4)



In [143]:
dict_mean_framewise['sub-10159']

0.16319031478878412

In [144]:
json_meas_PearsonCorrelation_relmat = {
    "Measure": "Pearson correlation",
    "MeasureDescription": "Pearson correlation",
    "Weighted": False,
    "Directed": False,
    "ValidDiagonal": True,
    "StorageFormat": "Full",
    "NonNegative": "",
    "Code": "https://github.com/pbergeret12/HalfPipe2Bids/tree/main"
}


output_filename = 'meas-PearsonCorrelation_relmat.json'
output_file = os.path.join(output_path, output_filename)

# Exporter le JSON
with open(output_file, 'w') as f:
    json.dump(json_meas_PearsonCorrelation_relmat, f, indent=4)

print(f"JSON exporté vers {output_file}")

JSON exporté vers ../draft/output/BEP017/meas-PearsonCorrelation_relmat.json


In [146]:
json_dataset_description = {
    "BIDSVersion": "1.9.0",
    "License": None,
    "Name": None,
    "ReferencesAndLinks": [],
    "DatasetDOI": None,
    "DatasetType": "derivative",
    "GeneratedBy": [
        {
            "Name": "Halfpipe2Bids",
            "Version": "0.1",
            "CodeURL": "https://github.com/pbergeret12/HalfPipe2Bids/tree/main"
        }
    ],
    "HowToAcknowledge": "Please refer to our repository: https://github.com/pbergeret12/HalfPipe2Bids/tree/main."
}

output_filename = 'dataset_description.json'
output_file = os.path.join(output_path, output_filename)

# Exporter le JSON
with open(output_file, 'w') as f:
    json.dump(json_dataset_description, f, indent=4)

print(f"JSON exporté vers {output_file}")


JSON exporté vers ../draft/output/BEP017/dataset_description.json
